In [2]:
import kagglehub

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler    
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVC

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import os

#https://www.kaggle.com/datasets/prathamtripathi/drug-classification
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import PIL 
from PIL import Image


In [3]:
models = torchvision.models.resnet18(pretrained=True)

/Users/amannindra/Projects/DSS/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/amannindra/Projects/DSS/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/amannindra/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)>

In [ ]:
train_data = []
train_folder = os.path.join(os.getcwd(), 'data', 'train_features')
l = os.listdir(train_folder)

dataframe = pd.read_csv(os.path.join(os.getcwd(), 'data', 'train_labels.csv'))
dataframe.head()

id_list = dataframe['id'].tolist()
for i in id_list:
    img = Image.open(os.path.join(train_folder, i + '.jpg'))
    train_data.append(img)
    
dataframe['image'] = train_data


In [ ]:
dataframe['image'] = [np.asarray(img) for img in dataframe['image']]

In [7]:

dataframe.head()


,id,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent,image
0,ZJ000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[[[140, 140, 140], [139, 139, 139], [139, 139,..."
1,ZJ000001,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"[[[131, 131, 131], [131, 131, 131], [132, 132,..."
2,ZJ000002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[[[158, 158, 158], [142, 142, 142], [139, 139,..."
3,ZJ000003,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"[[[70, 115, 112], [72, 117, 114], [75, 115, 11..."
4,ZJ000004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"[[[18, 37, 33], [92, 111, 107], [148, 167, 163..."


In [8]:
dataframe.iloc[0]['image'].shape

(540, 960, 3)